In [7]:
# Importar las librerías necesarias para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings

# Ignorar advertencias (warnings) para mantener una salida limpia
warnings.filterwarnings("ignore")

# Cargar el dataframe limpio.
df_cleaned = pd.read_csv('datos/df_cleaned.csv')

# Haremos una copia para mantener el original intacto.
df_featured = df_cleaned.copy()

### Section 5: Feature Engineering

#### Resumen Ejecutivo de la Sección

El propósito de esta sección es preparar el conjunto de datos para el modelado mediante la creación de nuevas características y la selección de las más relevantes. Basado en los hallazgos del EDA, la tarea principal aquí es transformar la columna categórica y multi-etiqueta `CONTAMINANTES` en un formato numérico que el algoritmo de clustering pueda utilizar.

La estrategia adoptada es **One-Hot Encoding**. Se creará una nueva columna binaria (0 o 1) para cada tipo de contaminante único. Esto convierte la información de texto en una representación numérica que permite al modelo identificar la presencia o ausencia de contaminantes específicos como factores de agrupamiento.

In [8]:
# 1. Extraer todas las etiquetas de contaminantes únicas
# Dividimos la columna 'CONTAMINANTES' por la coma, expandimos a una serie larga,
# eliminamos espacios en blanco y obtenemos los valores únicos.
unique_contaminants = df_featured['CONTAMINANTES'].str.split(', ', expand=True).stack().str.strip().unique()

# Mapeo de abreviaturas a nombres más legibles para las nuevas columnas
contaminant_short_names = {
    'Sin Contaminantes': 'NONE', 'DT': 'DT', 'FLUO': 'FLUO', 'COND': 'COND',
    'CF': 'CF', 'NO3': 'NO3', 'FE': 'FE', 'MN': 'MN', 'AS': 'AS', 'CD': 'CD',
    'CR': 'CR', 'HG': 'HG', 'PB': 'PB', 'ALC': 'ALC', 'SDT_ra': 'SDT_ra',
    'SDT_salin': 'SDT_salin'
}

# 2. Aplicar One-Hot Encoding
# Iteramos sobre cada contaminante único (excepto 'Sin Contaminantes')
new_cols_count = 0
for contaminant in unique_contaminants:
    if contaminant != 'Sin Contaminantes':
        short_name = contaminant_short_names.get(contaminant, contaminant)
        col_name = f"CONTAMINANTE_{short_name}"
        # Creamos una nueva columna. Será 1 si el string del contaminante está presente, 0 si no.
        df_featured[col_name] = df_featured['CONTAMINANTES'].apply(lambda x: 1 if contaminant in x else 0)
        new_cols_count += 1

# 3. Eliminar la columna original 'CONTAMINANTES'
df_featured = df_featured.drop(columns=['CONTAMINANTES'])

print(f"Se han creado {new_cols_count} nuevas columnas de contaminantes mediante one-hot encoding.")
print("La columna original 'CONTAMINANTES' ha sido eliminada.")
print(f"Dimensiones del DataFrame después de la ingeniería de características: {df_featured.shape}")

# 4. Mostrar el resultado
print("\n--- Primeras 5 filas del DataFrame con las nuevas características ---")
display(df_featured.head())

Se han creado 126 nuevas columnas de contaminantes mediante one-hot encoding.
La columna original 'CONTAMINANTES' ha sido eliminada.
Dimensiones del DataFrame después de la ingeniería de características: (1068, 181)

--- Primeras 5 filas del DataFrame con las nuevas características ---


,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,"CONTAMINANTE_ALC,DT,MN,","CONTAMINANTE_CONDUC,SDT_ra,SDT_salin,DT,MN,FE,","CONTAMINANTE_ALC,CONDUC,DT,MN,NO3,","CONTAMINANTE_ALC,NO3,","CONTAMINANTE_CONDUC,DT,MN,FE,","CONTAMINANTE_CR,MN,","CONTAMINANTE_FLUO,CR,FE,","CONTAMINANTE_CR,NI,FE,","CONTAMINANTE_AS,CR,","CONTAMINANTE_ALC,DT,NO3,"
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,0,0,0,0,0,0,0,0,0,0
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,0,0,0,0,0,0,0,0,0,0
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,0,0,0,0,0,0,0,0,0,0
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,0,0,0,0,0,0,0,0,0,0
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,0,0,0,0,0,0,0,0,0,0
